# Exercise 04

## Overview

1. RDKit
2. Docking

This tutorial is based on [rdkit-tutorials](https://github.com/rdkit/rdkit-tutorials/tree/master) and [LABODOCK](https://github.com/RyanZR/labodock).

## RDKit

[RDKit](https://www.rdkit.org/) is a software suite for cheminformatics, computational chemistry, and predictive modeling

A chemical curation pipeline has been developed using the open source toolkit RDKit. 

The [ChEMBL](https://www.ebi.ac.uk/chembl/) database is one of a number of public databases that contain bioactivity data on small molecule compounds curated from diverse sources. Incoming compounds are typically not standardised according to consistent rules. In order to maintain the quality of the final database and to easily compare and integrate data on the same compound from different sources it is necessary for the chemical structures in the database to be appropriately standardised. The solution: RDKit. RDKit is a open source tookit for the chemical curation pipeline.



### Basis steps with RDKit 

#### Reading and writing molecules

In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on colab')
  !pip install rdkit

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
# uncomment this if you try the tutorial and end up with low-quality images
# IPythonConsole.ipython_useSVG=True

#### Working with SMILES

If you have a SMILES string, the easiest thing to use is MolFromSmiles:

In [ ]:
m = Chem.MolFromSmiles('COc1ccc2c(c1)[nH]c(n2)[S@@](=O)Cc1ncc(c(c1C)OC)C')
m

Note that the coordinates used for the drawing are not present in the molecule, the RDKit generates them when the molecule is drawn.

#### Reading Mol file data

In [ ]:
molblock = """phenol
  Mrv1682210081607082D          

  7  7  0  0  0  0            999 V2000
   -0.6473    1.0929    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.3618    0.6804    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -1.3618   -0.1447    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
   -0.6473   -0.5572    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0671   -0.1447    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0671    0.6804    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.7816    1.0929    0.0000 O   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0  0  0  0
  2  3  2  0  0  0  0
  3  4  1  0  0  0  0
  4  5  2  0  0  0  0
  5  6  1  0  0  0  0
  1  6  2  0  0  0  0
  6  7  1  0  0  0  0
M  END
"""
m = Chem.MolFromMolBlock(molblock)
m

Here the molecule has coordinates that were read in from the Mol block. We can see this because the molecule has a conformer:

In [ ]:
m.GetNumConformers()

The conformer that is present is 2D (we can see that from the coordinates above):

In [ ]:
m.GetConformer().Is3D()

Here's another Mol block and the corresponding molecule:

In [ ]:
molblock2 ='''cyclopropanol-3D
  Mrv1682210081607153D          

 10 10  0  0  0  0            999 V2000
    0.9159    1.2233    0.2286 C   0  0  1  0  0  0  0  0  0  0  0  0
   -0.0738    1.2748   -0.9428 C   0  0  2  0  0  0  0  0  0  0  0  0
   -0.3167    0.3104    0.2232 C   0  0  2  0  0  0  0  0  0  0  0  0
   -1.3658    0.6623    1.1190 O   0  0  0  0  0  0  0  0  0  0  0  0
    1.8730    0.7413    0.0155 H   0  0  0  0  0  0  0  0  0  0  0  0
    0.8835    2.0739    0.9139 H   0  0  0  0  0  0  0  0  0  0  0  0
    0.2747    0.8266   -1.8737 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.7163    2.1557   -0.9817 H   0  0  0  0  0  0  0  0  0  0  0  0
   -0.1264   -0.7446    0.0137 H   0  0  0  0  0  0  0  0  0  0  0  0
   -1.1210    0.2486    1.9735 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0  0  0  0
  2  3  1  0  0  0  0
  1  3  1  0  0  0  0
  3  4  1  0  0  0  0
  1  5  1  0  0  0  0
  1  6  1  0  0  0  0
  2  7  1  0  0  0  0
  2  8  1  0  0  0  0
  3  9  1  0  0  0  0
  4 10  1  0  0  0  0
M  END
'''
m2 = Chem.MolFromMolBlock(molblock2)
m2

The odd coordinates in the drawing is because the molecule has a 3D conformer:

In [ ]:
m2.GetConformer().Is3D()

Notice that no Hs were drawn even though there are clearly H atoms in the Mol block. The default behavior for the RDKit Mol block reader is to remove Hs from the input molecule. If we want to keep the Hs and their coordinates, this behavior can be turned off:

In [ ]:
m2_h = Chem.MolFromMolBlock(molblock2,removeHs=False)
m2_h

#### RDKit: substructure queries using SMARTS

This notebook explores using `RDKit` to find matching 2D structures in organic molecules, using SMARTS queries.  It uses RDKit's very nice structure renderer to show matches.  This notebook isn't a replacment for the [rdkit documentation](http://www.rdkit.org/docs/), but is intended to complement it.

In [ ]:
from __future__ import print_function
# import rdkit components
from rdkit import rdBase
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

# use IPythonConsole for pretty drawings
from rdkit.Chem.Draw import IPythonConsole

## The next line is commented out 
### because GitHub does not render svg's embedded in notebooks
# IPythonConsole.ipython_useSVG=True

#### Defining and viewing a small set of example molecules

In [ ]:
# some heterocycles 
naphthalene = Chem.MolFromSmiles('c12ccccc1cccc2')
benzoxazole = Chem.MolFromSmiles('n1c2ccccc2oc1')
indane = Chem.MolFromSmiles('c1ccc2c(c1)CCC2')
skatole = Chem.MolFromSmiles('CC1=CNC2=CC=CC=C12')
benzene = Chem.MolFromSmiles('c1ccccc1')
quinoline = Chem.MolFromSmiles('n1cccc2ccccc12')

my_molecules = [naphthalene, 
                benzoxazole,
                indane,
                skatole,
                benzene,
                quinoline,
               ]

Our test molecules for this notebook will be the arbitrary set of heterocycles and homocycles (and benzene) above.  

It's easy to get a look at the structure of these molecules.

In [ ]:
Draw.MolsToGridImage(my_molecules, useSVG=False)

For `RDKit` molecules created via SMILES, ring properties are instantiated automatically on molecule creation.  Thus, the code below that tests whether any atom in each molecule is a member of exactly two SSSR rings works right off the bat.  As expected, all molecules have (at least one atom that is in) two rings, except for the fifth molecule, benzene.

In [ ]:
is_in_two_rings = Chem.MolFromSmarts('[R2]')
[mol.HasSubstructMatch(is_in_two_rings) for mol in my_molecules]

#### Defining substructure queries

[SMARTS](http://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) is a chemical structure query language.  RDKit can import SMARTS queries for use in searching against sets of existing RDKit molecules. [Here's a link](https://rdkit.org/docs/RDKit_Book.html#smarts-support-and-extensions) to the documentation about the RDKit's SMARTS support and the extensions it includes.

In [ ]:
# SMARTS query for any five-membered ring fused to a benzene ring
benzo_five = Chem.MolFromSmarts('[*r5R1]1[cR2]2[cR1][cR1][cR1][cR1][cR2]2[*r5R1][*r5R1]1')

In [ ]:
# SMARTS query for any six-membered ring fused to a benzene ring
benzo_six = Chem.MolFromSmarts('[*r6R1]1[cR2]2[cR1][cR1][cR1][cR1][cR2]2[*r6R1][*r6R1][*r6R1]1')

In [ ]:
smarts_queries = [benzo_five, benzo_six]

Using SMARTS instead of SMILES to initialize molecules does not result in instantiation of a molecule's RingInfo property.  Thus, until `Chem.GetSSSR()` is invoked on a SMARTS-derived molecule, certain queries will fail.

In [ ]:
# initialize rings
[Chem.GetSSSR(mol) for mol in smarts_queries]

# now it works
try:
    print([mol.HasSubstructMatch(is_in_two_rings) for mol in smarts_queries])
    print('Success!  Your query worked.\n')
except RuntimeError as err:
    print('ERROR; {0} because RingInfo wasn\'t initialized'.format(err))

Now that our query molecules' `RingInfo` is instantiated, it will be possible to compute meaningful 2D sets of coordinates for the SMARTS molecule and to both (a) query against the SMARTS molecule and (b) use the SMARTS molecule as a query against other molecules.  Without calling `Chem.GetSSSR()`, only the second option would be viable.

In [ ]:
[AllChem.Compute2DCoords(mol) for mol in smarts_queries]

In [ ]:
# matching the first query
for mol in my_molecules:
    AllChem.GenerateDepictionMatching2DStructure(mol, benzo_five, acceptFailure = True)

#### Showing the hits: five-membered rings fused to benzene

Our matching to our first query is done, and now all we have to do is draw the results.

In [ ]:
highlight_lists = [mol.GetSubstructMatch(benzo_five) for mol in my_molecules]
Draw.MolsToGridImage(my_molecules, 
                     highlightAtomLists = highlight_lists,
                     useSVG=False
                    )

#### RDKit pandas support
This is a quick tutorial will show some of the ways you can use RDKit together with pandas.

In [ ]:
from __future__ import print_function
%matplotlib inline

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

# The next line is commented out 
# because GitHub does not render svg's embedded in notebooks
IPythonConsole.ipython_useSVG=False

Load the table of drugs (downloaded from [ChEMBL](https://www.ebi.ac.uk/chembl/) )

This table of drugs contains approximatelly ~15k drugs in different phases of clinical trials.
The information includes:
- Chembl ID
- Name
- Synonyms (associated with the drug)
- Phase (clinical trial phase)
- if Passes the Rule of Five (Ro5)
- and the structure in SMILES format


In [ ]:
if 'google.colab' in str(get_ipython()):
  print('Running on colab')
  !wget https://raw.githubusercontent.com/yerkoescalona/structural_bioinformatics/main/ex04/chembl_drugs.txt.gz
else:
  print('Not running on colab.')
  print('You should have chembl_drugs.txt.gz in your path!')

In [ ]:
df = pd.read_csv('chembl_drugs.txt.gz', sep=';')
df.head()

In [ ]:
df.columns

In [ ]:
len(df)

In [ ]:
df = df[df['Smiles'].notnull() & # Keep cpds with SMILES
        (df['Phase'] == 4)] # are on the market
df.head()

Table contains CANONICAL_SMILES with SMILES which we can convert to RDKit molecules (default name ROMol)

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='Smiles')

Remove rows where RDKit failed to generate a molecule from SMILES

In [ ]:
df = df[~df['ROMol'].isnull()]

Extract a name from SYNONYMS column by applying a row-wise operation

In [ ]:
df['name'] = df.apply(lambda x: x['Synonyms'].split('|')[0] if type(x['Synonyms']) is str else None, axis=1)

Depict first 12 mols

In [ ]:
PandasTools.FrameToGridImage(df.head(12), legendsCol='name', molsPerRow=4)

Calculate some descriptors and visualize distributions

In [ ]:
from rdkit.Chem import Descriptors

In [ ]:
df['MW'] = df['ROMol'].map(Descriptors.MolWt)
df['logP'] = df['ROMol'].map(Descriptors.MolLogP)

In [ ]:
df['MW'].hist(bins=100);

In [ ]:
df['logP'].hist(bins=100);

In [ ]:
df[['MW', 'logP']].describe()

Do a substructure search on a dataframe

In [ ]:
query = Chem.MolFromSmarts('Nc1ccc(S(=O)(=O)-[*])cc1')
query

Notice that unspecified bonds in the SMARTS are "single or aromatic" queries. We could make this query a bit more specific:

In [ ]:
query = Chem.MolFromSmarts('N-c1ccc(-S(=O)(=O)-[*])cc1')
query

We do a substruture search by using the operator `>=` on a molecule column.

Here's an example where we do the search and count the number of matching rows:

In [ ]:
len(df[df['ROMol'] >= query])

RDKit by defaults highlights the matched substructures

In [ ]:
df[df['ROMol'] >= query][['Synonyms', 'ROMol']].head()

Just display the matching molecules:

In [ ]:
PandasTools.FrameToGridImage(df[df['ROMol'] >= query].head(12), legendsCol='name', molsPerRow=4)

Save the table as SD file

In [ ]:
PandasTools.WriteSDF(df, 'approved_drugs.sdf', idName='Parent Molecule', properties=df.columns)

RDKit Exercise:

1. Create new subset of the dataframe with the following properties:
- Use the file `new_chembl_drugs.txt.gz` as input

```python
new_df = pd.read_csv('new_chembl_drugs.txt.gz', sep=';')
```

- Make a selection of molecules that pass the Rule of Five (Rule of Five == 1)
- Make a selection of molecules that are in at least Phase 3 (Phase >= 3)
- Make a selection of all molecules that contain a benzene ring
- Save the subset as a SDF file with the name `new_approved_drugs.sdf`

2. With your new subset of molecules:
- Describe the distribution of the molecular weight and logP. 
- Explain the difference between the distributions with the previous results.


## Autodock Vina

[AutoDock Vina](https://vina.scripps.edu/) is a popular docking program within the AutoDock Suite due to its ease of use, speed, and open-source nature. It is widely used and has been modified by research groups worldwide.


### LABODOCK

In order to use Autodock Vina, we will use [LABODOCK](https://github.com/RyanZR/labodock).
LABODOCK is a collection of Colab Notebooks that provides a simple web interface for Docking.

**Open** the [Basic Molecular Docking](https://colab.research.google.com/github/RyanZR/labodock/blob/main/notebooks/basic_molecular_docking.ipynb) Notebook and follow the instructions. We will execute the default example.

### Default Example: ERα Ligand Binding Domain in complex with Raloxifene 

The default tutorial uses the protein with PDB ID [7KBS](https://www.rcsb.org/structure/7KBS).
This is a Estrogen Receptor Alpha (ERα) Ligand Binding Domain in complex with Raloxifene (See in [more details](https://www.rcsb.org/3d-view/7KBS?preset=ligandInteraction&label_asym_id=C)).

Context: When estrogen binds to the Estrogen Receptor Alpha (ERα), the receptor undergoes a conformational change, which allows it to bind to specific DNA sequences called estrogen response elements (EREs). The genes activated or repressed by this complex can lead to various cellular responses. For instance, in breast tissue, estrogen-ERα interaction can stimulate cell division and growth, which is a normal process but can also contribute to the development and progression of breast cancer in estrogen-receptor-positive tumors. 

Solution: Raloxifene is used as therapy.  When Raloxifene binds to the LBD of ERα, it induces a specific conformational change in the receptor. This change differs from the conformation induced by estrogen binding, which is critical for the drug's mechanism of action. Raloxifene acts as an antagonist in breast tissue, meaning it blocks the activity of estrogen.

![villin](./protein-ligand.png)
**Figure**. ERα Ligand Binding Domain in complex with Raloxifene 


### Docking Exercises:

1. Describe all the interactions between the ligand and the protein.
2. Use estrogen as a ligand instead of Raloxifene. What are the differences in the interactions between the ligand and the protein?
3. Using those difference, could you be able to predict the conformational change of the protein when binding to Raloxifene or estrogen?

# For the Project

If your proteins can potentially interact with a ligand you can use the following workflow:
1. Find the Smiles format of the ligand in the ChEMBL database.
2. Use your PDB id and ligand Smiles format for the docking.
3. Describe the interactions between the ligand and the protein.

If your protein doesn't have a ligand, you can use the following workflow:
1. Use one of the following molecules as a ligand to your protein:

- Formaldehyde or Acetaldehyde: These small aldehyde molecules can be used to show how different functional groups (like the aldehyde group) interact with proteins. They can illustrate polar interactions and the role of hydrogen bonding in docking.

- Glycerol: This molecule, with its three hydroxyl groups, can demonstrate polyol interactions with proteins. It's particularly relevant for showing hydrogen bonding and the effect of multiple binding sites.

- Acetic Acid or Propionic Acid: These small carboxylic acids can be used to discuss the importance of charge in protein-ligand interactions, showing how deprotonation and protonation states affect docking.

- Benzene or Toluene: These aromatic compounds can illustrate hydrophobic interactions and pi-stacking, which are important in the docking of many drugs and natural ligands.

- Nitrate or Phosphate Ions: As small inorganic ions, they can be used to teach about ionic interactions and the role of electrostatics in protein-ligand docking.

- Urea or Thiourea: These molecules can be used to show how amide and thiol groups interact with proteins, and they're particularly interesting for discussing solvation effects and hydrogen bonding.

2. Use your PDB id and ligand Smiles format for the docking.
3. Describe the interactions between the ligand and the protein.
